In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


# 100 Recs - D8

## 1 - Corridor Improvements

In [2]:
speed_dict = {"mean_speed_mph": 16}
trip_dict = {"pct_trips_competitive": 0.10}

corridor_gdf_d8 = ((better_bus_utils.select_transit_routes_corridor_improvements(
    speed_dict = speed_dict,
    trip_dict = trip_dict)
                   )
                   >>filter(_.caltrans_district=="08 - San Bernardino")
                  )                    

In [3]:
corridor_gdf_d8

,calitp_itp_id,route_id,category,geometry,service_date,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district
59,269,23,intersects_shn,"LINESTRING (-117.15193 33.52372, -117.15370 33...",2022-05-04,19.0,25.0,3,15.418561,medium,61.8,3.0,0.12,35.976,1.9996,08 - San Bernardino
60,269,30,intersects_shn,"LINESTRING (-117.22945 33.78494, -117.22946 33...",2022-05-04,6.0,10.0,3,13.604556,medium,46.8,1.0,0.10,30.780,1.6400,08 - San Bernardino


In [4]:
#corridor_gdf >> count(_.caltrans_district)

In [5]:
corridor_d8 = (corridor_gdf_d8
               >> select(-_.service_date)
              )

corridor_d8.explore("mean_speed_mph", legend=True, tiles="CartoDB positron", cmap=ZERO_THIRTY_COLORSCALE)

In [6]:
# for each transit operator, generate rtfiltermapper
D8_filtermapper = rt_filter_map_plot.from_gcs(itp_id=269, analysis_date=dt.date(2022,5,4))
D8_filtermapper.rt_trips >> head(5)

found parquet


,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,route_type,median_time,direction,mean_speed_mph,calitp_agency_name
0,269,0,2022-05-04,20373467858498656,1808030,15,0,shp-15-58,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,18:06:01,Westbound,12.297201,Riverside Transit Agency
1,269,0,2022-05-04,-5290092726997443108,273030,15,0,shp-15-59,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,11:45:51,Westbound,12.746440,Riverside Transit Agency
2,269,0,2022-05-04,5684670834086266506,1312030,15,0,shp-15-59,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,15:53:31,Westbound,12.488653,Riverside Transit Agency
3,269,0,2022-05-04,-7229066287274309249,1331030,15,0,shp-15-58,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,16:36:57,Westbound,11.909406,Riverside Transit Agency
4,269,0,2022-05-04,-2221390301129503368,802030,15,0,shp-15-59,2022-01-04,2099-01-01,15,Riverside/Downtown-Merced,None,3,10:28:32,Westbound,13.873231,Riverside Transit Agency


In [7]:
# get small df of route_id and route_short_name from rt_trips
D8_routenames = (D8_filtermapper.rt_trips
                 >> count(_.route_id, _.route_short_name, _.route_long_name)
                 >> select(-_.n)
                )

D8_routenames >> head(5)

,route_id,route_short_name,route_long_name
0,1,1,W. Corona-UC Riverside
1,10,10,Riverside/Watkins-Galleria
2,11,11,Moreno Valley-March ARB
3,12,12,Corona Hills Plaza/Riverside/La Cadena-Merced
4,13,13,Hunter Park Metro-Galleria


In [8]:
# inner join w gdf of slow routes
corridor_d8_names = (corridor_d8
                     >> inner_join(_,D8_routenames)
                    )

corridor_d8_names

,calitp_itp_id,route_id,category,geometry,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district,route_short_name,route_long_name
0,269,23,intersects_shn,"LINESTRING (-117.15193 33.52372, -117.15370 33...",19.0,25.0,3,15.418561,medium,61.8,3.0,0.12,35.976,1.9996,08 - San Bernardino,23,Temecula - Murrieta - Wildomar
1,269,30,intersects_shn,"LINESTRING (-117.22945 33.78494, -117.22946 33...",6.0,10.0,3,13.604556,medium,46.8,1.0,0.10,30.780,1.6400,08 - San Bernardino,30,Weston & Carter-Perris Station- May Ranch


In [9]:
# pass the route names as a filter parameter
routename_list_c = corridor_d8_names["route_short_name"].tolist()
routename_list_c

['23', '30']

In [10]:
D8_filtermapper.set_filter(start_time='06:00', end_time='09:00', route_names = routename_list_c)

In [11]:
# make maps etc
m = D8_filtermapper.segment_speed_map()
m

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [12]:
D8_filtermapper.set_filter(start_time='15:00', end_time='19:00', route_names = routename_list_c)

In [13]:
# make maps etc
m = D8_filtermapper.segment_speed_map()
m

/opt/conda/lib/python3.10/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [14]:
# output detailed_map_view, fitered to slow segments (p20 speed below certain cutoff (8-12mph?))

## 2 - Hotspot Improvements

In [15]:
hotspot_gdf_d8 = ((better_bus_utils.get_sorted_transit_routes(recommendation_category="hotspot"))
                   >>filter(_.caltrans_district=="08 - San Bernardino")
                   >>head(5) #get top 5
                  )

In [16]:
hotspot_gdf_d8

,calitp_itp_id,route_id,category,geometry,service_date,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district
885,269,32,intersects_shn,"LINESTRING (-116.96957 33.80801, -116.96963 33...",2022-05-04,19.0,26.0,3,10.635828,medium,17.4,0.0,0.0,32.146154,2.140769,08 - San Bernardino
886,269,24,intersects_shn,"LINESTRING (-117.10707 33.47908, -117.10722 33...",2022-05-04,15.0,22.0,3,10.889040,medium,27.0,0.0,0.0,35.045455,1.973636,08 - San Bernardino
887,269,51,intersects_shn,"LINESTRING (-117.33975 33.97793, -117.33989 33...",2022-05-04,9.0,16.0,3,11.265268,short,0.0,0.0,0.0,12.600000,1.780000,08 - San Bernardino
888,269,21,other,"LINESTRING (-117.45960 33.91186, -117.45950 33...",2022-05-04,12.0,20.0,3,11.588145,short,6.0,0.0,0.0,16.170000,2.285000,08 - San Bernardino
889,269,55,intersects_shn,"LINESTRING (-117.15422 33.53759, -117.15411 33...",2022-05-04,6.0,7.0,3,12.027029,short,10.2,0.0,0.0,26.057143,3.067143,08 - San Bernardino


In [25]:
hotspot_gdf_d8 = (hotspot_gdf_d8
               >> select(-_.service_date)
              )

hotspot_gdf_d8.explore("mean_speed_mph", legend=True, tiles="CartoDB positron", cmap=ZERO_THIRTY_COLORSCALE)

In [18]:
# inner join w gdf of slow routes
hotspot_d8_names = (hotspot_gdf_d8
                     >> inner_join(_,D8_routenames)
                    )

hotspot_d8_names

,calitp_itp_id,route_id,category,geometry,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district,route_short_name,route_long_name
0,269,32,intersects_shn,"LINESTRING (-116.96957 33.80801, -116.96963 33...",19.0,26.0,3,10.635828,medium,17.4,0.0,0.0,32.146154,2.140769,08 - San Bernardino,32,Hemet Valley Mall - Mount San Jacinto College
1,269,24,intersects_shn,"LINESTRING (-117.10707 33.47908, -117.10722 33...",15.0,22.0,3,10.889040,medium,27.0,0.0,0.0,35.045455,1.973636,08 - San Bernardino,24,Promenade Mall - Temecula Valley Hospital
2,269,51,intersects_shn,"LINESTRING (-117.33975 33.97793, -117.33989 33...",9.0,16.0,3,11.265268,short,0.0,0.0,0.0,12.600000,1.780000,08 - San Bernardino,51,UCR - Canyon Crest Towne Centre
3,269,21,other,"LINESTRING (-117.45960 33.91186, -117.45950 33...",12.0,20.0,3,11.588145,short,6.0,0.0,0.0,16.170000,2.285000,08 - San Bernardino,21,Galleria-Pedley
4,269,55,intersects_shn,"LINESTRING (-117.15422 33.53759, -117.15411 33...",6.0,7.0,3,12.027029,short,10.2,0.0,0.0,26.057143,3.067143,08 - San Bernardino,55,Temecula Trolley


In [19]:
# pass the route names as a filter parameter
routename_list_hs = hotspot_d8_names["route_short_name"].tolist()
routename_list_hs

['32', '24', '51', '21', '55']

In [20]:
D8_filtermapper.reset_filter()

In [21]:
D8_filtermapper.set_filter(start_time='06:00', end_time='09:00', route_names = routename_list_hs)

In [22]:
# make maps etc
m = D8_filtermapper.segment_speed_map()
m

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [23]:
D8_filtermapper.set_filter(start_time='15:00', end_time='19:00', route_names = routename_list_hs)

In [24]:
# make maps etc
m = D8_filtermapper.segment_speed_map()
m

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

